In [3]:
import os
import pickle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# -------- CONFIG --------
LOG_DIR = "../obs_logs"
N = 20  # Show first N by time

# -------- FIND & SORT FILES --------
def extract_timestamp(filename):
    # Assumes format: observations_YYYYMMDD_HHMMSS.pkl
    try:
        return filename.split("_")[1].replace(".pkl", "")
    except IndexError:
        return ""

pkl_files = [
    # os.path.join(LOG_DIR, f)
    # for f in os.listdir(LOG_DIR)
    # if f.startswith("observations_") and f.endswith(".pkl")
    # "/home/xichen6/Documents/repos/genie_sim_v2.1/genie_sim/action_logs/observations_20250723_094830.pkl",
    # "/home/xichen6/Documents/repos/genie_sim_v2.1/genie_sim/action_logs/observations_20250723_093542.pkl",
    "/home/xichen6/Documents/repos/genie_sim_v2.3/genie_sim_v2.3-xc/AgiBot-World-Submission/CogACT/obs_logs/default_task/iter_1/observations_20250811_043141.pkl",
]

# Sort by timestamp string
pkl_files.sort(key=lambda x: extract_timestamp(os.path.basename(x)))

# Take first N
pkl_files = pkl_files[:N]

# -------- IMAGE LOADING --------
def get_rgb_image(file_path):
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    # cam_data = data['camera']['/G1/link_pitch_head/Head_Camera_01']
    # img_np_flat = cam_data['rgb_camera']
    # img_np = img_np_flat.reshape((480, 640, 4))[:, :, :3]
    
    img_np_left = data['images']['head_left']
    img_np_head = data['images']['cam_top']
    img_np_right = data['images']['head_right']
    
    # Combine images into a single RGB image
    img_np = np.concatenate((img_np_left, img_np_head, img_np_right), axis=1)
    
    if img_np.shape[0] == 3:
        img_np = np.transpose(img_np, (1, 2, 0))  # CHW to HWC
    if img_np.dtype != np.uint8:
        img_np = (img_np * 255).clip(0, 255).astype(np.uint8)
    print(f"shape: {img_np.shape}, dtype: {img_np.dtype}")
    return img_np

# -------- VISUALIZATION --------
def show_image(idx):
    img_np = get_rgb_image(pkl_files[idx])
    plt.figure(figsize=(8, 6))
    plt.imshow(img_np)
    plt.axis("off")
    plt.title(f"{idx}: {os.path.basename(pkl_files[idx])}")
    plt.show()

# -------- SLIDER UI --------
if pkl_files:
    interact(
        show_image,
        idx=IntSlider(min=0, max=len(pkl_files)-1, step=1, value=0, description='Frame')
    )
else:
    print("No observation files found.")


interactive(children=(IntSlider(value=0, description='Frame', max=0), Output()), _dom_classes=('widget-interac…